In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv


## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import re
import string
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
# from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import ngrams

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from keras.optimizers import SGD, Adam, Adagrad, Adadelta, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

In [4]:
from warnings import filterwarnings
filterwarnings('ignore')

sns.color_palette('dark')
%matplotlib inline

tqdm.pandas()

## Load Data

In [5]:
dftrain = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
dftest = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample_submission = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [6]:
dftrain.shape, dftest.shape

((2834, 6), (7, 4))

## Clean data

In [7]:
def clean_text(sentence):
    # remove numbers
    pattern = re.compile(r'[0-9]+')
    sentence = sentence.lower()
    sentence = pattern.sub(' ', sentence).strip()
    # remove punctuations
    newSentence = ''
    for char in sentence:
        if char not in string.punctuation:
            newSentence += char
    # Tokenize
    word_list = word_tokenize(newSentence)
    # stop words
    stopwords_list = set(stopwords.words('english'))
    # remove stop words
    word_list = [word for word in word_list if word not in stopwords_list]
    # stemming
    ps  = PorterStemmer()
    word_list = [ps.stem(word) for word in word_list]
    # list to sentence
    sentence = ' '.join(word_list)
    
    return sentence

In [8]:
dftrain['clean_text'] = dftrain['excerpt'].progress_apply(clean_text)
dftest['clean_text'] = dftest['excerpt'].apply(clean_text)

100%|██████████| 2834/2834 [00:14<00:00, 198.26it/s]


## Split data

In [9]:
Xtrain, Xcv, Ytrain, Ycv = train_test_split(dftrain['clean_text'], dftrain['target'], random_state=21, test_size=0.2)

## Tokenize and pad sequences

In [10]:
# Tokenize our training data
tokenizer = Tokenizer(num_words=20000, oov_token='<UNK>')
tokenizer.fit_on_texts(Xtrain)

# word to number and vice versa map
word2num = tokenizer.word_index
num2word = {k:w for w, k in word2num.items()}

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences(Xtrain)

# Get max training sequence length
maxlen = max([len(x) for x in train_sequences])

# Pad the training sequences
train_padded = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=210)


valid_sequences = tokenizer.texts_to_sequences(Xcv)
valid_padded = pad_sequences(valid_sequences, padding='post', truncating='post', maxlen=210)

test_sequences = tokenizer.texts_to_sequences(dftest['clean_text'])
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen=210)

In [11]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [12]:
inp = Input(shape=(210, ))
x = Embedding(20000, 256, trainable=False)(inp)
x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.1,recurrent_dropout=0.1))(x)
x = Attention()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="linear")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 210)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 210, 256)          5120000   
_________________________________________________________________
bidirectional (Bidirectional (None, 210, 600)          1336800   
_________________________________________________________________
attention (Attention)        (None, 600)               810       
_________________________________________________________________
dense (Dense)                (None, 256)               153856    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257   

In [13]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
file_path = 'model.hdf5'
ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor="val_loss", mode="auto", patience=1)

In [14]:
model.fit(train_padded, Ytrain, batch_size=1024, epochs=30, validation_data=(valid_padded, Ycv), callbacks=[ckpt])

Epoch 1/30
3/3 [==============================] - 16s 3s/step - loss: 1.8750 - mse: 1.8750 - val_loss: 1.2574 - val_mse: 1.2574

Epoch 00001: val_loss improved from inf to 1.25736, saving model to model.hdf5
Epoch 2/30
3/3 [==============================] - 8s 3s/step - loss: 1.6337 - mse: 1.6337 - val_loss: 1.2946 - val_mse: 1.2946

Epoch 00002: val_loss did not improve from 1.25736
Epoch 3/30
3/3 [==============================] - 8s 3s/step - loss: 1.1990 - mse: 1.1990 - val_loss: 1.5034 - val_mse: 1.5034

Epoch 00003: val_loss did not improve from 1.25736
Epoch 4/30
3/3 [==============================] - 8s 3s/step - loss: 1.3394 - mse: 1.3394 - val_loss: 1.3829 - val_mse: 1.3829

Epoch 00004: val_loss did not improve from 1.25736
Epoch 5/30
3/3 [==============================] - 8s 3s/step - loss: 1.2011 - mse: 1.2011 - val_loss: 1.2186 - val_mse: 1.2186

Epoch 00005: val_loss improved from 1.25736 to 1.21865, saving model to model.hdf5
Epoch 6/30
3/3 [============================

## Prediction

In [15]:
test_pred = model.predict(test_padded)

In [16]:
test_pred = test_pred.reshape(test_pred.shape[0])

In [17]:
submission = pd.DataFrame({'id': dftest['id'], 'target':test_pred})

In [18]:
submission.to_csv('submission.csv', index=False)